In [1]:
import sys

sys.path.insert(0, "/home/elina/PycharmProjects/capstone_project")
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from torchmetrics.classification import BinaryAUROC
from modules.model.main import ConvCatNet
from modules.data.preprocessing import PreProcess

import torch
import tenseal as ts
import numpy as np

from modules.encryption.keys import create_ctx
def size_is_ok():
    img = preprocessor.cuda_image_array.copy()[0][0]

    conv1_weight = net.conv1.weight.data.view(
        -1, net.conv1.kernel_size[0],
        net.conv1.kernel_size[1]
    ).tolist()
    conv1_bias = net.conv1.bias.data.tolist()
    nk = img.tolist().copy()[0]

    x_enc_0, windows_nb_0 = ts.im2col_encoding(
        context, nk, 3,
        3, 1
    )
    y = x_enc_0.conv2d_im2col(conv1_weight[0], windows_nb_0) + conv1_bias[0]
    print(y)

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 20  # poly degree /2 should be bigger than image to columns for conv size
context, server_context = create_ctx(bits_scale=30,num_mul=7)  # scale up as working with floats,


In [35]:
preprocessor = PreProcess(image_size=image_size)
train_dataloader, test_dataloader = preprocessor.preprocess()

  0%|          | 2/1920 [00:00<09:44,  3.28it/s]


In [36]:
net = ConvCatNet(image_size=image_size).to(device)
print(net)

ConvCatNet(
  (lrelu): LeakyReLU(negative_slope=0.1)
  (conv1): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(3, 1, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=324, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


In [37]:
size_is_ok()

In [38]:
EPOCHS = 100
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
bauc = BinaryAUROC(thresholds=None)

In [39]:
def model_train_loop(net, train_dataloader):
    running_loss = 0.0
    predictions = []
    ground_truth = []

    for idx, data in enumerate(train_dataloader, 0):
        # get the inputs
        # print(i,data)
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=10)

        optimizer.step()
        running_loss += loss.item()

        predictions.append(outputs.flatten())
        ground_truth.append(labels.flatten())

    # net.to('cpu')
    # plot_grad_flow(net.named_parameters())
    # plt.show()
    # net.to(device)

    predictions = torch.cat(predictions)
    ground_truth = torch.cat(ground_truth)

    healthy_f1 = f1_score((predictions > 0.5).float().cpu(), ground_truth.cpu(), pos_label=0)
    rocauc = bauc(predictions, ground_truth)
    print(
        '[%d, %5d] loss: %.3f, ROC: %.3f, F1_healthy: %.3f\n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))

In [40]:
def model_test_loop(net, test_dataloader):
    with torch.no_grad():
        running_loss = 0.0
        test_predictions = []
        test_ground_truth = []

        for idx, data in enumerate(test_dataloader, 0):
            inputs, labels = data

            outputs = net(inputs)
            loss = criterion(outputs.flatten(), labels)
            running_loss += loss.item()

            test_predictions.append(outputs.flatten())
            test_ground_truth.append(labels.flatten())

        test_predictions = torch.cat(test_predictions)
        test_ground_truth = torch.cat(test_ground_truth)

        healthy_f1 = f1_score((test_predictions > 0.5).float().cpu(), test_ground_truth.cpu(), pos_label=0)
        rocauc = bauc(test_predictions, test_ground_truth)
        print('[%d, %5d] Validation loss: %.3f, ROC: %.3f, F1_healthy: %.3f \n' % (
            epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))


In [41]:
# f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)

In [42]:
# print(len(train_dataloader))

In [43]:

# MODEL learns the trivial case of all 1s, which means it has capacity to learn:

for epoch in range(2):  # loop over the dataset multiple times

    model_train_loop(net, train_dataloader)
    model_test_loop(net, test_dataloader)


# print('Finished Training')

[1,     1] loss: 0.722, ROC: 0.000, F1_healthy: 0.000

[1,     2] Validation loss: 1.438, ROC: 0.000, F1_healthy: 0.000 

[2,     1] loss: 0.721, ROC: 0.000, F1_healthy: 0.000

[2,     2] Validation loss: 1.433, ROC: 0.000, F1_healthy: 0.000 



/home/elina/anaconda3/envs/capstone_project/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/elina/anaconda3/envs/capstone_project/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/elina/anaconda3/envs/capstone_project/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/elina/anaconda3/envs/capstone_project/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false posit

# enc

In [44]:
class EncConvNet:

    def __init__(self, torch_nn):
        self.conv1_weight = torch_nn.conv1.weight.data.view(
            -1, torch_nn.conv1.kernel_size[0],
            torch_nn.conv1.kernel_size[1]
        ).tolist()
        self.conv1_bias = torch_nn.conv1.bias.data.tolist()

        self.conv2_weight = torch_nn.conv2.weight.data.view(
            -1, torch_nn.conv2.kernel_size[0],
            torch_nn.conv2.kernel_size[1]
        ).tolist()
        self.conv2_bias = torch_nn.conv2.bias.data.tolist()

        self.fc1_weight = torch_nn.fc1.weight.T.data.tolist()
        self.fc1_bias = torch_nn.fc1.bias.data.tolist()

        self.fc2_weight = torch_nn.fc2.weight.T.data.tolist()
        self.fc2_bias = torch_nn.fc2.bias.data.tolist()


    def forward(self, enc_x, windows_nb,windows_nb_2=0):
        # conv layer
        enc_channels = 0
        for ind, kernel in enumerate(self.conv1_weight):
            y = enc_x[ind].conv2d_im2col(kernel, windows_nb) + self.conv1_bias[0]

            print("one channel")
            print(y.shape)
            enc_channels+=y
            # enc_channels.append(y)

        # enc_x_conv_1 = ts.CKKSVector.pack_vectors(enc_channels)
        print("the conv shape")
        print(enc_channels.shape)

        # enc_channels_2 = 0
        # for ind, kernel in enumerate(self.conv2_weight):
        #     y = enc_x[ind+3].conv2d_im2col(kernel, windows_nb_2) + self.conv2_bias[0]
        #
        #     print("one channel")
        #     print(y.shape)
        #     enc_channels_2+=y
        #
        # enc_x_conv_2 = ts.CKKSVector.pack_vectors([enc_channels,enc_channels_2])
        # print(enc_x_conv_2.shape)

        # pack all channels into a single flattened vector
        # enc_x = ts.CKKSVector.pack_vectors(enc_channels)
        enc_x = enc_channels
        # fc1 layer
        enc_x = enc_x.mm(self.fc1_weight) + self.fc1_bias
        # square activation
        enc_x = EncConvNet.relu(enc_x)
        # fc2 layer
        enc_x = enc_x.mm(self.fc2_weight) + self.fc2_bias
        enc_x = EncConvNet.sigmoid(enc_x)
        return enc_x

    @staticmethod
    def sigmoid(enc_x):
        # We use the polynomial approximation of degree 3
        # sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
        # from https://eprint.iacr.org/2018/462.pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.5, 0.197, 0, -0.004])

    @staticmethod
    def relu(enc_x):
        # We use the polynomial approximation of degree 3
        # relu(x) = 0.47+0.5*x+0.09*(x**2)-1.7*e-10*(x**3)
        # from https://openreview.net/attachment?id=rkxsgkHKvH&name=original_pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.47, 0.5, 0.09, -0.0000000007])

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [45]:
enc_net = EncConvNet(net)

In [46]:
for idx, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    print("one conv1")
    print(inputs.shape)
    print(net.conv1(inputs).shape)
    print(net.conv2(inputs).shape)
    outputs = net(inputs)
    print(outputs)

one conv1
torch.Size([1, 3, 20, 20])
torch.Size([1, 1, 18, 18])
torch.Size([1, 1, 16, 16])
tensor([[0.4870]], grad_fn=<SigmoidBackward0>)
one conv1
torch.Size([1, 3, 20, 20])
torch.Size([1, 1, 18, 18])
torch.Size([1, 1, 16, 16])
tensor([[0.4898]], grad_fn=<SigmoidBackward0>)


In [52]:
for i in range(2):
    img = preprocessor.cuda_image_array[i][0]
    kernel_shape = net.conv1.kernel_size

    data_0 = img[0]
    data_1 = img[1]
    data_2 = img[2]
    print(data_1.shape)
    x_enc_0, windows_nb_0 = ts.im2col_encoding(
        context, data_0, kernel_shape[0],
        kernel_shape[1], 1
    )
    x_enc_1, windows_nb_1 = ts.im2col_encoding(
        context, data_1, kernel_shape[0],
        kernel_shape[1], 1
    )
    x_enc_2, windows_nb_2 = ts.im2col_encoding(
        context, data_2, kernel_shape[0],
        kernel_shape[1], 1
    )


    x_enc_1_0, windows_nb_1_0 = ts.im2col_encoding(
        context, data_0, 5,
        5, 1
    )
    x_enc_1_1, windows_nb_1_1 = ts.im2col_encoding(
        context, data_1, 5,
       5, 1
    )
    x_enc_1_2, windows_nb_1_2 = ts.im2col_encoding(
        context, data_2, 5,
        5, 1
    )
    x_enc = (x_enc_0, x_enc_1, x_enc_2,   x_enc_1_0, x_enc_1_1 ,x_enc_1_2)
    enc_output = enc_net(x_enc, windows_nb_1, windows_nb_1_2)
    print(enc_output.decrypt())

torch.Size([20, 20])
one channel
[324]
one channel
[324]
one channel
[324]
the conv shape
[324]
[0.4841976302148786]
torch.Size([20, 20])
one channel
[324]
one channel
[324]
one channel
[324]
the conv shape
[324]
[0.48422946235151393]
